In [1]:
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder

In [2]:
dateColumnNames = ['contact_date','Glycemie_der_date','HbA1c_der_date','der_date_poids','der_date_taille','first_contact_date']

dfView = pd.read_csv('PatientsHTA.zip',nrows=1)
df = pd.read_csv('PatientsHTA.zip',engine='c',parse_dates=dateColumnNames)

# Suppression des lignes trop peu nombreuses

Nous souhaitons faire un apprentissage en utilisant la dimension temporelle comme filtre pour le CNN. Pour ça il faut donc que nous ayons plusieurs entrée. Avant de commencer à traîter les données nous supprimons donc toutes les lignes qui n'ont pas plusieurs entrées de ```person_id```. Nous choisissons arbitrairement que pour être utile à l'apprentissage, il faut au moins 10 entrées dans cette colonne

In [4]:
valueCounts = df.person_id.value_counts()
dfEnought = df[df.person_id.isin(valueCounts[valueCounts.values >= 10].index)]

# Suppression des colonnes innutiles

## Suppression de ```Age_now```

Nous pouvons supprimer la colonne ```Age_now``` car les données qu'elle contient sont identique à celles de la colonne ```year_of_birth```

In [5]:
dfWithoutAgeNow = dfEnought.drop('Age_now', axis='columns')

## Suppression de ```contact_id```

In [6]:
dfWithoutContactID = dfWithoutAgeNow.drop('contact_id',axis='columns')

## Suppression des noms de médicaments

In [7]:
dfGroupedByMoleculeLabel = dfWithoutContactID.groupby('product_atc_code')[['molecule_label','short_name','long_name','Classe','product_atc']].count()
dfGroupedByMoleculeLabel

,molecule_label,short_name,long_name,Classe,product_atc
product_atc_code,,,,,
C02AC06,2276,2276,2276,2276,2276
C03BX03,655,655,655,655,655
C03CA01,402,402,402,402,402
C03DA01,86,86,86,86,86
C03DA04,10,10,10,10,10
C03EA04,3,3,3,3,3
C03EB01,10,10,10,10,10
C07AB03,3380,3380,3380,3380,3380
C07AB04,34,34,34,34,34


Nous voyons que les différentes colonnes de noms de médicaments sont identiques, nous pouvons donc n'en garder qu'une seule. Nous choisirons de garder ```product_atc_code```

In [8]:
dropColumnNames = dfGroupedByMoleculeLabel.columns.to_list()
dfWithATCCode = dfWithoutContactID.drop(dropColumnNames, axis='columns')

In [9]:
dfWithATCCode

,person_id,specialty_label,contact_date,cip,dosage_1,dose_1,dose_2,product_atc_code,box,quantity,...,HbA1c_der_date,HbA1c_der_mesure,gender_code,Age_presc,year_of_birth,Poids,der_date_poids,Taille,der_date_taille,first_contact_date
8,27746613.0,Médecin généraliste,2013-05-15,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,...,NaT,NaN,M,93.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
9,27746613.0,Médecin généraliste,2014-12-19,3.400960e+12,300.0,16800.0,56.0,C09XA02,4.0,1.00,...,NaT,NaN,M,94.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
12,27746613.0,Médecin généraliste,2013-06-14,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,...,NaT,NaN,M,93.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
24,27746613.0,Médecin généraliste,2013-01-10,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,...,NaT,NaN,M,93.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
32,27746613.0,Médecin généraliste,2013-12-30,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,...,NaT,NaN,M,93.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50621,14936234.0,Médecin généraliste,2016-09-11,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,1.00,...,NaT,NaN,M,81.0,1935.0,63.8,2017-07-27,162.0,2011-11-07,2002-09-24
50622,14936234.0,Médecin généraliste,2016-11-16,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,1.00,...,NaT,NaN,M,81.0,1935.0,63.8,2017-07-27,162.0,2011-11-07,2002-09-24
50625,26636453.0,Médecin généraliste,2016-11-16,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.75,...,2017-10-03,5.60,M,79.0,1937.0,83.9,2018-02-21,NaN,NaT,2012-02-14
50627,18889430.0,Médecin généraliste,2013-02-01,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.50,...,2017-05-15,8.01,M,85.0,1928.0,NaN,NaT,NaN,NaT,2007-12-02


## Suppression des colonnes ```'*der*'```

Les colonnes ```*der*``` contiennent la dernière données. Cette donnée peut être récupérée grâce à la date de la visite et à aux valeurs mesurées. Par exemple, il n'est pas nécessaire d'avoir une colonne ```der_date``` ou ```der_mesure```. Les données de ces deux types de colonnes peuvent être récupéré grâce à la ligne qui correspond à la dernière date de la mesure, que l'on peut trouver grâce à la colonne ```contact_date```

In [24]:
derColumnNames = []

for c in dfWithATCCode.columns:
    if ('der_date' in c) or ('der_mesure' in c):
        derColumnNames.append(c)

dfWithoutDer = dfWithATCCode.drop(derColumnNames,axis='columns')

# Traîtement des données

## Temps entre chaque visite

Nous ajoutons une colonne contenant le temps entre chaque visite. Le CNN serait sûrement caoable de trouver cette information tout seul, mais cela nous permet d'enlevver 2 colonnes et donc de diminuer le nombre d'hyper paramètres à apprendre. Les valeurs seront un offset depuis la première date de visite

In [85]:
visit_time_offset = dfWithoutDer.contact_date - dfWithoutDer.first_contact_date
dfWithVisitOffset = dfWithoutDer.drop(['contact_date','first_contact_date'],axis='columns')
dfWithVisitOffset['visit_time_offset'] = visit_time_offset

## Conversion des données

### Conversion des dates en nombre

In [86]:
dfWithVisitOffset

,person_id,specialty_label,cip,dosage_1,dose_1,dose_2,product_atc_code,box,quantity,frequency_label,...,Tension Diastolique,Tension Systolique,Glycemie_prescription,HbA1c_prescription,gender_code,Age_presc,year_of_birth,Poids,Taille,visit_time_offset
8,27746613.0,Médecin généraliste,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,Jour,...,125.0,70.0,NaN,NaN,M,93.0,1920.0,75.0,166.0,210 days
9,27746613.0,Médecin généraliste,3.400960e+12,300.0,16800.0,56.0,C09XA02,4.0,1.00,Jour,...,135.0,75.0,NaN,NaN,M,94.0,1920.0,75.0,166.0,793 days
12,27746613.0,Médecin généraliste,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,Jour,...,130.0,73.0,NaN,NaN,M,93.0,1920.0,75.0,166.0,240 days
24,27746613.0,Médecin généraliste,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,Jour,...,148.0,77.0,NaN,NaN,M,93.0,1920.0,75.0,166.0,85 days
32,27746613.0,Médecin généraliste,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,Jour,...,130.0,80.0,NaN,NaN,M,93.0,1920.0,75.0,166.0,439 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50621,14936234.0,Médecin généraliste,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,1.00,Jour,...,112.0,61.0,NaN,NaN,M,81.0,1935.0,63.8,162.0,5101 days
50622,14936234.0,Médecin généraliste,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,1.00,Jour,...,108.0,61.0,NaN,NaN,M,81.0,1935.0,63.8,162.0,5167 days
50625,26636453.0,Médecin généraliste,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.75,Jour,...,120.0,60.0,NaN,NaN,M,79.0,1937.0,83.9,NaN,1737 days
50627,18889430.0,Médecin généraliste,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.50,Jour,...,120.0,70.0,NaN,NaN,M,85.0,1928.0,NaN,NaN,1888 days


In [87]:
# On encode en nombre de jours pour avoir des valeurs plus simple à lire pour nous autre, pauvre humains qui avons besoin de pouvoir interpréter les données
dfWithVisitOffset.visit_time_offset = dfWithVisitOffset.visit_time_offset.dt.total_seconds() / (3600 * 24)

### Encodage des valeurs non numériques

In [92]:
specialtyEncoder = LabelEncoder()
ATCCodeEncoder = LabelEncoder()
genderEncoder = LabelEncoder()
frequencyLabelEncoder = LabelEncoder()

dfWithVisitOffset.specialty_label = specialtyEncoder.fit_transform(dfWithVisitOffset.specialty_label)
dfWithVisitOffset.product_atc_code = ATCCodeEncoder.fit_transform(dfWithVisitOffset.product_atc_code)
dfWithVisitOffset.frequency_label = frequencyLabelEncoder.fit_transform(dfWithVisitOffset.frequency_label.astype(str))
dfWithVisitOffset.gender_code = ATCCodeEncoder.fit_transform(dfWithVisitOffset.gender_code)


# Prédiction

In [93]:
dfWithVisitOffset.reset_index(inplace=True,drop=True)
dfWithVisitOffset

,person_id,specialty_label,cip,dosage_1,dose_1,dose_2,product_atc_code,box,quantity,frequency_label,...,Tension Diastolique,Tension Systolique,Glycemie_prescription,HbA1c_prescription,gender_code,Age_presc,year_of_birth,Poids,Taille,visit_time_offset
0,27746613.0,2,3.400960e+12,300.0,16800.0,56.0,29,3.0,1.00,0,...,125.0,70.0,NaN,NaN,1,93.0,1920.0,75.0,166.0,210.0
1,27746613.0,2,3.400960e+12,300.0,16800.0,56.0,29,4.0,1.00,0,...,135.0,75.0,NaN,NaN,1,94.0,1920.0,75.0,166.0,793.0
2,27746613.0,2,3.400960e+12,300.0,16800.0,56.0,29,3.0,1.00,0,...,130.0,73.0,NaN,NaN,1,93.0,1920.0,75.0,166.0,240.0
3,27746613.0,2,3.400960e+12,300.0,16800.0,56.0,29,3.0,1.00,0,...,148.0,77.0,NaN,NaN,1,93.0,1920.0,75.0,166.0,85.0
4,27746613.0,2,3.400960e+12,300.0,16800.0,56.0,29,3.0,1.00,0,...,130.0,80.0,NaN,NaN,1,93.0,1920.0,75.0,166.0,439.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19228,14936234.0,2,3.400940e+12,500.0,15000.0,30.0,2,1.0,1.00,0,...,112.0,61.0,NaN,NaN,1,81.0,1935.0,63.8,162.0,5101.0
19229,14936234.0,2,3.400940e+12,500.0,15000.0,30.0,2,1.0,1.00,0,...,108.0,61.0,NaN,NaN,1,81.0,1935.0,63.8,162.0,5167.0
19230,26636453.0,2,3.400940e+12,500.0,15000.0,30.0,2,1.0,0.75,0,...,120.0,60.0,NaN,NaN,1,79.0,1937.0,83.9,NaN,1737.0
19231,18889430.0,2,3.400940e+12,500.0,15000.0,30.0,2,1.0,0.50,0,...,120.0,70.0,NaN,NaN,1,85.0,1928.0,NaN,NaN,1888.0
